# 背景

对指定个股和指数进行30分钟级别的监控，包括趋势、反转指标等，并通过语音进行报告。

In [5]:
! export HTTPS_PROXY=""
from alpha.notebook import *
from alpha.core.rsi_stats import rsiday, rsi30
from alpha.ml.index_sh_pv import IndexShPeakValleys
import datetime
import itertools
import traceback
from IPython.display import display
from omicron.core.numpy_extensions import find_runs
await init_notebook()

g = {
    "notified": set(),
}

#shpv30m = IndexShPeakValleys(inference_mode=True)

In [6]:
async def notify(frame, text, voice_only=False):
    if hasattr(frame, 'hour'):
        key = f"{frame.hour:02d}:{frame.minute:02d} {text}"
    else:
        key = f"{frame} {text}"
    if key in g["notified"]:
        return
    
    if not voice_only:
        print(key)
        
    g["notified"].add(key)
    say(text)
    
async def rsi_check_30m(code):
    sec = Stock(code)
    name = sec.display_name
    
    mbars = await get_bars(code, 60, '30m')
    end = mbars["frame"][-1]
    
    close = mbars["close"]
    r30si = relative_strength_index(close, period=6)
    p30rsi = rsi30.get_proba(code, r30si[-1])
    
    # 30分钟的顶底背离，需要有一次处在高位/低位
#     flag, _ = divergency(r30si[18:], close[18:])
#     if flag > 0:
#         await notify(end, f"{name}出现{abs(flag)}次30分钟底背离")
#     elif flag < 0:
#         await notify(end, f"{name}出现{abs(flag)}次30分钟顶背离")
        
    dbars = await get_bars(code, 60, '1d')
    close = dbars["close"]
    drsi = relative_strength_index(close, period=6)
    pdrsi = np.array([rsiday.get_proba(code, r) for r in drsi[-2:]])
    
    if p30rsi > 0.93:
        if any(pdrsi > 0.93):
            await notify(end, f"{name}日线及30分钟高位运行，警惕高位反转风险")
        else:
            await notify(end, f"{name}*30分钟*高位运行,高位概率{p30rsi:.0%}，请适度保持轻仓")
    elif p30rsi is None and r30si > 93:
        await notify(end, f"{name}*30分钟*高位运行,高位概率{r30si:.0f}，请适度保持轻仓")
    
    if p30rsi < 0.07:
        if any(pdrsi < 0.07):
            await notify(end, f"{name}日线及30分钟低位运行，留意低位反转机会")
        else:
            await notify(end, f"{name}*30分钟*低位运行,低位概率{1 - p30rsi:.0%}，留意低位反转机会")
    elif p30rsi is None and r30si < 10:
        await notify(end, f"{name}*30分钟*低位运行,高位概率{r30si:.0f}，请适度保持轻仓")
            
async def rsi_check_day(code):
    sec = Stock(code)
    name = sec.display_name
    
    bars = await get_bars(code, 60)
    close = bars["close"]
    end = bars[-1]["frame"]
    
    rsi = relative_strength_index(close)[18:]
    
    #判断顶底背离，只有当其处于高位/低位时更有意义
#     flag, _ = divergency(rsi, close[18:], min(len(rsi), 40))
#     if flag > 0:
#         await notify(end, f"{name}出现{abs(flag)}次日线底背离")
#     if flag < 0:
#         await notify(end, f"{name}出现{abs(flag)}次日线顶背离")
        
    prsi = np.array([rsiday.get_proba(code, r) for r in rsi[-2:]])
    if np.all(prsi) and any(prsi > 0.93):
        await notify(end, f"{name}两日内出现高位，顶部概率为{max(prsi):.0%}")
    elif not prsi[0] and any(rsi[-2:] > 93):
        await notify(end, f"{name}两日内出现高位，{np.round(rsi[-2:],0)}")
    if np.all(prsi) and any(prsi < 0.07):
        await notify(end, f"{name}两日内出现低位，底部概率为{max(1 - prsi):.0%}")
    elif not prsi[0] and any(rsi[-2:] < 7):
        await notify(end, f"{name}两日内出现低位：{np.round(rsi[-2:],0)}")
        
    rsi = prsi if np.all(prsi) else rsi[-2:]
    return name, code, *rsi

In [7]:
async def do_check(holdings:List[str], check_at=None):
    end = arrow.now() if check_at is None else arrow.get(check_at)
    start = tf.shift(tf.floor(end, FrameType.MIN30), -39, FrameType.MIN30)
    
    for code in ("000001.XSHG", *holdings):
        try:
            await rsi_check_30m(code)
        except Exception:
            pass
    
async def mon(holdings, check_at=None):
    import asyncio
    
    holdings = [name_to_code(name) for name in holdings]
    
    now = arrow.now()
    wakeup_time = []
    for tk in tf.ticks[FrameType.MIN5]:
        hour = tk//60
        minute = tk%60

        wakeup_time.append(arrow.Arrow(now.year, now.month, now.day, hour, minute, tzinfo="Asia/Shanghai"))
        
    if check_at:
        wakeup_time.insert(0, arrow.now().shift(seconds=5))
        
    end = tf.floor(arrow.now(), FrameType.DAY)
    
    data = []
    for code in ("000001.XSHG", *holdings):
        name, code, prsi1, prsi0 = await rsi_check_day(code)

        data.append([name, code, prsi1, prsi0])
        
    df = pd.DataFrame(data, columns=["name","code", "rsi1", "rsi0"])
    display(df)
    
    for tm in wakeup_time:
        if arrow.now() > tm:
            continue
            
        seconds = (tm.timestamp() - arrow.now().timestamp())
        await asyncio.sleep(seconds)
        print(f"=========== {tm.hour}:{tm.minute:02d}============")
        await do_check(holdings, check_at)

In [9]:
holdings = [
    "和科达",
    "国联证券",
    "华策影视",
    "智度股份"
]
await mon(holdings, arrow.now().shift(seconds=3))
print("一天监测结束")

,name,code,rsi1,rsi0
0,上证指数,000001.XSHG,0.35,0.32
1,和科达,002816.XSHE,0.46,0.25
2,国联证券,601456.XSHG,56.53,64.02
3,华策影视,300133.XSHE,0.39,0.39
4,智度股份,000676.XSHE,0.64,0.64


=========== 9:05============
15:00 华策影视*30分钟*低位运行,低位概率95%，留意低位反转机会
=========== 9:35============
=========== 9:40============
=========== 9:45============
=========== 9:50============
=========== 9:55============
=========== 10:00============
=========== 10:05============
=========== 10:10============
=========== 10:15============
=========== 10:20============
=========== 10:25============
=========== 10:30============
10:30 和科达*30分钟*低位运行,低位概率94%，留意低位反转机会
10:30 智度股份*30分钟*低位运行,低位概率94%，留意低位反转机会
=========== 10:35============
=========== 10:40============
10:40 智度股份*30分钟*低位运行,低位概率95%，留意低位反转机会
=========== 10:45============
10:45 智度股份*30分钟*低位运行,低位概率96%，留意低位反转机会
=========== 10:50============
10:50 智度股份*30分钟*低位运行,低位概率96%，留意低位反转机会
=========== 10:55============
10:55 智度股份*30分钟*低位运行,低位概率97%，留意低位反转机会
=========== 11:00============
11:00 和科达*30分钟*低位运行,低位概率94%，留意低位反转机会
11:00 智度股份*30分钟*低位运行,低位概率97%，留意低位反转机会
=========== 11:05============
11:05 智度股份*30分钟*低位运行,低位概率98%，留意低位反转机会
=========== 11:10===========